In [10]:
#Import lIBRARIES
import pandas as pd
import numpy as np
import os as os

In [11]:
# example making new folder with os

os.makedirs('Data/',exist_ok=True) 
# Confirm folder created
os.listdir("Data/")

['Final_basics.csv.gz',
 'final_tmdb_data_2001.csv.gz',
 'title.akas.csv.gz',
 'title.ratings.csv.gz',
 'title_basics.csv.gz',
 'tmdb_api_results_2001.json']

In [12]:
##Load Basics set and assign to DF

basics_url="https://datasets.imdbws.com/title.basics.tsv.gz"

basics = pd.read_csv(basics_url, sep='\t', low_memory=False)

#basics.to_csv("Data/title_basics.csv.gz",compression='gzip',index=False)

In [13]:
#Assign local file to DF
#basics = pd.read_csv("Data/title_basics.csv.gz", low_memory=False)
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [14]:
#Basics preprocessing

#Replace "\N" with np.nan
basics.replace({'\\N': np.nan}, inplace=True) 
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,NaN,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,NaN,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,NaN,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,NaN,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,NaN,1,"Comedy,Short"


In [15]:
#Eliminate movies that are null for runtimeMinutes
basics = basics.dropna(subset=['genres', 'runtimeMinutes'])

In [16]:
basics.isna().sum()

tconst                  0
titleType               0
primaryTitle            0
originalTitle           0
isAdult                 0
startYear           35687
endYear           2313293
runtimeMinutes          0
genres                  0
dtype: int64

In [17]:
#keep only titleType==Movie
movie_filter = basics['titleType']=='movie'

basics=basics[movie_filter]

In [18]:
#keep startYear 2000-2022
start_filt = basics['startYear'] >= '2000'
start_filt.head()

570     False
587     False
672     False
1172    False
1273    False
Name: startYear, dtype: bool

In [19]:
end_filt = basics['startYear']<='2022'
end_filt.head()

570     True
587     True
672     True
1172    True
1273    True
Name: startYear, dtype: bool

In [20]:
basics = basics.loc[start_filt & end_filt, :]
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 211904 entries, 13082 to 8999025
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   tconst          211904 non-null  object
 1   titleType       211904 non-null  object
 2   primaryTitle    211904 non-null  object
 3   originalTitle   211904 non-null  object
 4   isAdult         211904 non-null  object
 5   startYear       211904 non-null  object
 6   endYear         0 non-null       object
 7   runtimeMinutes  211904 non-null  object
 8   genres          211904 non-null  object
dtypes: object(9)
memory usage: 16.2+ MB


In [21]:
#Eliminate movies that include  "Documentary" in genre (see tip below)
#basics = basics[basics["genres"].str.contains("Documentary") == False]
is_documentary = basics['genres'].str.contains('documentary',case=False)
basics = basics[~is_documentary]
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 140049 entries, 34805 to 8998924
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   tconst          140049 non-null  object
 1   titleType       140049 non-null  object
 2   primaryTitle    140049 non-null  object
 3   originalTitle   140049 non-null  object
 4   isAdult         140049 non-null  object
 5   startYear       140049 non-null  object
 6   endYear         0 non-null       object
 7   runtimeMinutes  140049 non-null  object
 8   genres          140049 non-null  object
dtypes: object(9)
memory usage: 10.7+ MB


In [22]:
basics['genres'].value_counts()

Drama                       34200
Comedy                      12935
Comedy,Drama                 6189
Horror                       5494
Drama,Romance                4101
                            ...  
Crime,Family,Thriller           1
Animation,Music,Sci-Fi          1
Drama,History,News              1
Action,Animation,History        1
Action,History,Western          1
Name: genres, Length: 957, dtype: int64

In [23]:
##Load AKAs data set and save locally 
akas_url="https://datasets.imdbws.com/title.akas.tsv.gz"

akas = pd.read_csv(akas_url, sep='\t', low_memory=False)

akas.to_csv("Data/title.akas.csv.gz",compression='gzip',index=False)

In [24]:
#Assign local file to DF
akas = pd.read_csv("Data/title.akas.csv.gz", low_memory=False)
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
3,tt0000001,4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
4,tt0000001,5,Карменсита,RU,\N,imdbDisplay,\N,0


In [25]:
akas.replace({'\\N':np.nan}, inplace=True)

In [26]:
#AKA - keep only US entries. I keep breaking the main df, I'm creating a dummy
#df that I will move back once done.
dfa= akas
dfa.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,NaN,imdbDisplay,NaN,0
1,tt0000001,2,Carmencita,DE,NaN,NaN,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,NaN,imdbDisplay,NaN,0
3,tt0000001,4,Καρμενσίτα,GR,NaN,imdbDisplay,NaN,0
4,tt0000001,5,Карменсита,RU,NaN,imdbDisplay,NaN,0


In [27]:
#Apply filter to retain only movies in US region
region_filt = dfa['region']=='US'
region_filt.head()

0    False
1    False
2    False
3    False
4    False
Name: region, dtype: bool

In [28]:
#Check that filtering worked...it did.  ONLY US region films are showing.
dfa[region_filt].head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
5,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0
14,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0
33,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0
36,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0
41,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0


In [29]:
dfa = dfa.loc[region_filt, :]
dfa.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
5,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0
14,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0
33,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0
36,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0
41,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0


In [30]:
dfa['region'].value_counts()

US    1328708
Name: region, dtype: int64

In [31]:
#Transfer dummy set back to actual df
akas = dfa
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
5,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0
14,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0
33,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0
36,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0
41,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0


In [32]:
#Save new cleansed df to local csv file from earlier 
akas.to_csv("Data/title.akas.csv.gz",compression='gzip',index=False)

In [33]:
#Display df info as requested
akas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1328708 entries, 5 to 32328026
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   titleId          1328708 non-null  object
 1   ordering         1328708 non-null  int64 
 2   title            1328708 non-null  object
 3   region           1328708 non-null  object
 4   language         3581 non-null     object
 5   types            1026140 non-null  object
 6   attributes       44187 non-null    object
 7   isOriginalTitle  1327333 non-null  object
dtypes: int64(1), object(7)
memory usage: 91.2+ MB


In [34]:
##Load Ratings data set and assign to DF
ratings_url="https://datasets.imdbws.com/title.ratings.tsv.gz"

ratings = pd.read_csv(ratings_url, sep='\t', low_memory=False)

ratings.to_csv("Data/title.ratings.csv.gz",compression='gzip',index=False)

In [35]:
#Assign local file to DF
ratings = pd.read_csv("Data/title.ratings.csv.gz", low_memory=False)
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1888
1,tt0000002,5.9,251
2,tt0000003,6.5,1679
3,tt0000004,5.8,164
4,tt0000005,6.2,2498


In [36]:
#RATINGS Preprocessing
ratings = ratings.replace({'\\N':np.nan}) 

In [37]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1254186 entries, 0 to 1254185
Data columns (total 3 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   tconst         1254186 non-null  object 
 1   averageRating  1254186 non-null  float64
 2   numVotes       1254186 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 28.7+ MB


In [38]:
ratings.to_csv("Data/title.ratings.csv.gz",compression='gzip',index=False)

In [39]:
del ratings

In [40]:
# Filter the basics table down to only include the US by using the filter akas dataframe
keepers =basics['tconst'].isin(akas['titleId'])


In [41]:
basics = basics[keepers]
basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 81403 entries, 34805 to 8998840
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   tconst          81403 non-null  object
 1   titleType       81403 non-null  object
 2   primaryTitle    81403 non-null  object
 3   originalTitle   81403 non-null  object
 4   isAdult         81403 non-null  object
 5   startYear       81403 non-null  object
 6   endYear         0 non-null      object
 7   runtimeMinutes  81403 non-null  object
 8   genres          81403 non-null  object
dtypes: object(9)
memory usage: 6.2+ MB


In [42]:
basics.to_csv("Data/Final_basics.csv.gz", compression= 'gzip',index=False)
